In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5
)


#### **Static Prompt**

In [2]:
response = model.invoke("Welche sprachen sprichst du?")
print(response.content)

Ich wurde von Google trainiert und spreche viele Sprachen. Ich kann Texte auf Deutsch verstehen und generieren.  Ich kann auch in anderen Sprachen kommunizieren, aber meine Fähigkeiten variieren je nach Sprache.  Meine Stärke liegt in der Verarbeitung und Generierung von Texten, nicht im direkten Sprechen.


#### **Dynamic Prompt Template**

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    You are a {profession}, who only talks about related subject.
    If someone asks you a question that is not related to your profession, you will answer with "I don't know".
    
    Answer the following question:
    {question}
    """,
    input_variables=["profession", "question"]
)

prompt_template = prompt.invoke(
    {
        "profession": "doctor",
        "question": "What is the capital of France?"
    }
)

response = model.invoke(prompt_template)
print(response.content)

I don't know.


##### **Save and Load Template**

In [5]:
prompt.save('template.json')

In [6]:
from langchain_core.prompts import load_prompt

template = load_prompt('template.json')

#### **Chat Prompt Template**

In [3]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ('system', "You are a {profession}, who only talks about related subject and says I don't know to non-related queries"),
    ('human', "{query}")
])

prompt = template.invoke(
    {
        'profession':'data-scientist', 
        'query':'when we use prompt template and chat prompt template just give oneliner'
    }
)

response = model.invoke(prompt)
print(response.content)


Prompt templates offer structure and control for generating consistent outputs from large language models, while chat prompt templates add conversational context.


#### **Message Placeholder**

- Use for loading the chat history as LLM context

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
def load_chats():
    """create custom function for loading the chats from database"""
    pass

template = ChatPromptTemplate([
    ('system', 'you are an e-commerce chatbot'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{query}')
])


while True:
    user_query = input("You: ")
    if user_query.lower() == 'exit':
        break
    
    prompt = template.invoke(
        {
            'chat_history':chat_history,
            'query': user_query
        }
    )
    response = model.invoke(prompt)

    # store chats using message-labels
    chat_history.append(HumanMessage(content=user_query))
    chat_history.append(AIMessage(content=response.content))
    
    print("AI: ", response.content)